In [ ]:
from datetime import datetime, timedelta
from feast import FeatureStore

from feast.infra.offline_stores.contrib.postgres_offline_store.postgres import PostgreSQLOfflineStoreConfig
from feast.infra.online_stores.redis import RedisOnlineStoreConfig
from feast.repo_config import RepoConfig, RegistryConfig

We start by setting up the Feast client and initializing a FeatureStore instance

In [ ]:
offline_store_config = PostgreSQLOfflineStoreConfig(
    host="postgresql-offline-store.default.svc.cluster.local",
    database="postgres",
    db_schema="public",
    user="postgres",
    password="postgres"
)

In [ ]:
online_store_config = RedisOnlineStoreConfig(
    connection_string="redis-online-store.default.svc.cluster.local:6379,username=default,password=redis,db=0"
)

In [ ]:
registry_config = RegistryConfig(
    registry_store_type="KubeflowRegistryStore",
    path="",
    project="kubeflow-user"
)

In [ ]:
repo_config = RepoConfig(
    project="kubeflow-user",
    registry=registry_config,
    provider="local",
    offline_store=offline_store_config,
    online_store=online_store_config
)

In [ ]:
fs = FeatureStore(config=repo_config, repo_path=None)

Then, we fetch all feature view definitions

In [ ]:
feature_views = fs.list_feature_views(allow_cache=True)

In [ ]:
fv_names = []

for fv in feature_views:
    print(fv.name, fv.entities, fv.schema, sep=" - ")
    fv_names.append(fv.name)

After choosing the feature views (their names to be specific) we want to materialize, we can use either the _materialize_incremental()_ or the _materialize()_ method.

The first method loads incremental new feature data up to the specified end time from either the specified feature views, or all feature views if none are specified, into the online store where it is available for online serving. The start time of the interval materialized is either the most recent end time of a prior materialization or (now - ttl) if no such prior materialization exists.

The second one loads feature data in the specified interval from either the specified feature views, or all feature views if none are specified, into the online store where it is available for online serving.

You can choose the method that fits your use case.

In [ ]:
fs.materialize_incremental(end_date=datetime.utcnow(), feature_views=fv_names)

In [ ]:
fs.materialize(
    start_date=datetime.utcnow() - timedelta(days=7), end_date=datetime.utcnow(), feature_views=fv_names
)